In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

# Đọc dữ liệu từ file CSV
df = pd.read_csv('transactions.csv')

# Chuyển đổi dữ liệu thành định dạng của Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'product_id', 'rating']], reader)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
trainset, testset = train_test_split(data, test_size=0.2)

In [2]:
# Tạo mô hình SVD
model = SVD()

# Huấn luyện mô hình trên tập huấn luyện
model.fit(trainset)

In [3]:
# Tạo dự đoán trên tập kiểm tra
predictions = model.test(testset)

# Đánh giá mô hình
accuracy.rmse(predictions)

# Hiển thị kết quả dự đoán
print("Predictions:")
for uid, iid, true_r, est, _ in predictions[:10]:
    print(f'User {uid:<10} - Product {iid:<10} | Actual: {true_r:<5.2}, Predicted: {est:<5.2f}')

RMSE: 0.2036
Predictions:
User 13963972   - Product 251773296  | Actual: 3.0  , Predicted: 3.14 
User 18911318   - Product 164545513  | Actual: 3.0  , Predicted: 2.91 
User 7769131    - Product 6982965    | Actual: 4.0  , Predicted: 3.74 
User 26282600   - Product 6879031    | Actual: 2.0  , Predicted: 2.18 
User 7042294    - Product 117030549  | Actual: 3.2  , Predicted: 3.24 
User 19687532   - Product 172474580  | Actual: 3.0  , Predicted: 3.00 
User 18964427   - Product 7958872    | Actual: 2.2  , Predicted: 2.57 
User 15630745   - Product 74823296   | Actual: 3.4  , Predicted: 3.26 
User 7626196    - Product 7958766    | Actual: 3.0  , Predicted: 2.93 
User 29178135   - Product 90483814   | Actual: 2.8  , Predicted: 2.78 


In [4]:
# Hàm gợi ý sản phẩm
def get_top_n_recommendations(user_id, n=10):
    # Lấy danh sách tất cả các product_id
    all_product_ids = df['product_id'].unique()
    
    # Dự đoán rating cho tất cả các sản phẩm mà user chưa đánh giá
    user_ratings = [(product_id, model.predict(user_id, product_id).est) for product_id in all_product_ids]
    
    # Sắp xếp theo rating dự đoán
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    
    # Lấy top n sản phẩm
    top_n_recommendations = user_ratings[:n]
    
    return top_n_recommendations

In [5]:
# Ví dụ: Gợi ý 10 sản phẩm cho user_id = 1
recommendations = get_top_n_recommendations(22580465, 10)
print("\nTop Recommendations:")
for product_id, rating in recommendations:
    print(f'Product ID: {product_id:<10}, Predicted Rating: {rating:<5.2f}')


Top Recommendations:
Product ID: 134025169 , Predicted Rating: 4.94 
Product ID: 18013163  , Predicted Rating: 4.93 
Product ID: 107709913 , Predicted Rating: 4.91 
Product ID: 29916656  , Predicted Rating: 4.89 
Product ID: 6306239   , Predicted Rating: 4.87 
Product ID: 102085482 , Predicted Rating: 4.86 
Product ID: 134025507 , Predicted Rating: 4.84 
Product ID: 52417141  , Predicted Rating: 4.84 
Product ID: 14847948  , Predicted Rating: 4.81 
Product ID: 172474728 , Predicted Rating: 4.80 


In [6]:
df['rating']

0        3.5
1        4.0
2        3.4
3        1.0
4        1.7
        ... 
16071    3.6
16072    5.0
16073    1.7
16074    4.3
16075    2.0
Name: rating, Length: 16076, dtype: float64

In [7]:
df.product_id.nunique()

505

In [8]:
# Ví dụ: Gợi ý 10 sản phẩm cho user_id = 1
recommendations = get_top_n_recommendations(16877862, 10)
for product_id, rating in recommendations:
    print(f'Product ID: {product_id}, Predicted Rating: {rating}')

Product ID: 197536148, Predicted Rating: 4.995181485243612
Product ID: 275225058, Predicted Rating: 4.992195849672041
Product ID: 121009977, Predicted Rating: 4.963666525890647
Product ID: 22543488, Predicted Rating: 4.924082753597729
Product ID: 134025169, Predicted Rating: 4.922938608707052
Product ID: 178322259, Predicted Rating: 4.874882994244405
Product ID: 172445145, Predicted Rating: 4.871554193833593
Product ID: 107709913, Predicted Rating: 4.866930390327042
Product ID: 29916656, Predicted Rating: 4.862563208895772
Product ID: 130287356, Predicted Rating: 4.861764345453031
